# Importation et lecture de fichier :

In [42]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# nltk.download("stopwords")
# nltk.download("punkt")
# nltk.download("averaged_perceptron_tagger")
# nltk.download('wordnet')

import numpy as np
import pandas as pd
from sklearn import preprocessing

import re
import string

import torch
from torch.nn import functional as F
from torch.autograd import Variable

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

ModuleNotFoundError: No module named 'torchtext'

In [12]:
 filepath = "data/eit_annot_100_1899.txt"
 with open(filepath, "r") as f:
    content = f.readlines()

info = [x.split(")")[0] for x in content]

# Get only the line where the author = author
idx_consensus = np.array([y.split(",")[2] for y in info])
idx_consensus = np.where(idx_consensus == "consensus")

results = np.array([y.split(",")[1] for y in info])[idx_consensus]

# tweets
tweets = np.array([x.split(")")[1] for x in content])[idx_consensus]
    
df = pd.DataFrame()
df['tweet'] = tweets
df['result'] = results

df.head()

,tweet,result
0,@melo_gabby Sao os assuntos mais comentados d...,irr
1,@Marbelle30 ...Esta familia de #twitter cada ...,irr
2,@apple please bring back the old Siri app for...,neu
3,Oef..#Microsoft trapt Google Toolbar uit #Sky...,neu
4,I ♥ @Apple http://t.co/a8on3IAa\n,pos


In [33]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    
    return url.sub('', text)

example = 'New competition launched: https://www.kaggle.com/c/nlp-getting-started'

remove_URL(example)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    
    return text.translate(table)

example = "I am #king"
print(remove_punct(example))

remove_emoji("Omg another Earthquake 😔😔")

wnl = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence_words = sentence.split(' ')
    new_sentence_words = list()
    
    for sentence_word in sentence_words:
        sentence_word = sentence_word.replace('#', '')
        new_sentence_word = wnl.lemmatize(sentence_word.lower(), wordnet.VERB)
        new_sentence_words.append(new_sentence_word)
        
    new_sentence = ' '.join(new_sentence_words)
    new_sentence = new_sentence.strip()
    
    return new_sentence


I am king


In [39]:
df['tweet_c'] = df['tweet'].apply(lambda x: remove_URL(x))
df['tweet_c'] = df['tweet_c'].apply(lambda x: remove_emoji(x))
df['tweet_c'] = df['tweet_c'].apply(lambda x: remove_punct(x))
df['tweet_c'] = df['tweet_c'].apply(lambda x: lemmatize_sentence(x))

In [40]:
df.head(10)

,tweet,result,tweet_c
0,@melo_gabby Sao os assuntos mais comentados d...,irr,melogabby sao os assuntos mais comentados do m...
1,@Marbelle30 ...Esta familia de #twitter cada ...,irr,marbelle30 esta familia de twitter cada vez cr...
2,@apple please bring back the old Siri app for...,neu,apple please bring back the old siri app for u...
3,Oef..#Microsoft trapt Google Toolbar uit #Sky...,neu,oefmicrosoft trapt google toolbar uit skype
4,I ♥ @Apple http://t.co/a8on3IAa\n,pos,i apple
5,RT @NeowinFeed: Steve Ballmer slams Android #...,neu,rt neowinfeed steve ballmer slam android micro...
6,Is #Google Down? http://t.co/wFrzKYsN via @Ej...,neu,be google down via ejunkie good for bing micr...
7,This is better than *every* magazine having t...,pos,this be better than every magazine have their ...
8,"RT @mpastrana: RT @apple no pasa nada, todo e...",irr,rt mpastrana rt apple no pasa nada todo esta b...
9,Many new features for Android. #NexusPrime #G...,neu,many new feature for android nexusprime google


NameError: name 'TEXT' is not defined